In [1]:
#https://www.youtube.com/watch?v=8ZZDNd4eyVI&t=1s&ab_channel=Mr.PSolver
import numpy as np
import sympy as smp
import pickle
filename = "newVariables.pickle"

Symbols Init

In [14]:
g = smp.symbols('g')
M1, M2 = smp.symbols('M1, M2')
L1, L2 = smp.symbols('L1, L2')
T1, T2 = smp.symbols('T1, T2')
Fx, Fy = smp.symbols('Fx, Fy')


KP, KD, desiredX, desiredY, desiredXdott, desiredYdott, currentX, currentY = smp.symbols('KP, KD, desiredX, desiredY, desiredXdott, desiredYdott, currentX, currentY')

th1, th2 = smp.symbols('th1, th2')
th1d, th2d = smp.symbols('th1d, th2d')
th1dd, th2dd = smp.symbols('th1dd, th2dd')

Dynamic Model

In [42]:

I1 = 1/12*L1*L1*M1
I2 = 1/12*L2*L2*M2

d1 = L1/2
d2 = L2/2

Fd = smp.Matrix([Fx, Fy]).T;

P1 = smp.Matrix([ - L2*smp.cos(th1 + th2) - L1*smp.cos(th1) ,
                   - L2*smp.sin(th1 + th2) - L1*smp.sin(th1) ])

P2 = smp.Matrix([ - L2*smp.cos(th1 + th2) ,
                   - L2*smp.sin(th1 + th2) ])

M = smp.Matrix([ [I2 + I1 + 2*M2*d2*L1*smp.cos(th2) + M2*L1*L1 + M2*d2*d2 + M1*d1*d1  , I2 + d2*d2 * M2 + M2*d2*L1*smp.cos(th2)] ,
                  [I2 + d2*M2*L1*smp.cos(th2) + M2*d2*d2                             , I2 + d2*d2 * M2                      ]])

B = smp.Matrix([ - d2*M2*L1*th2d*th2d*smp.sin(th2) - 2*M2*d2*L1*th1d*th2d*smp.sin(th2),
                    d2*M2*L1*smp.sin(th2)*th1d*th1d])

G = smp.Matrix([ - g*M2*d2*smp.sin(th1 + th2) - (M2*L1 + d1*M1)*g*smp.sin(th1) ,
                  - d2*g*M2*smp.sin(th1 + th2)                              ])

F = smp.Matrix([ Fd*P1 ,
      Fd*P2 ])

T = smp.Matrix([ T1 ,
                  T2 ])

thedd = M.inv() * ( - B - G + T + F)


(2, 1)

Gravity Compensation

In [47]:
Tformulas = smp.solve([thedd[0], thedd[1]], (T1, T2), simplify=True, rational=True)
Tformulas = [Tformulas[T1], Tformulas[T2]]

Impedance Control

In [48]:
J = smp.Matrix([[- L2*smp.cos(th1 + th2) - L1*smp.cos(th1), -L2*smp.cos(th1 + th2)],
                [- L2*smp.sin(th1 + th2) - L1*smp.sin(th1), -L2*smp.sin(th1 + th2)],
                [                                0,                  0]])


desiredPosition = smp.Matrix([desiredX, desiredY, 0 ])
desiredVelocity = smp.Matrix([desiredXdott, desiredYdott, 0])

thed = smp.Matrix([th1d, th2d])

currentPosition = smp.Matrix([currentX,  currentY, 0])

impedanceControl = J.T*(KP*(desiredPosition - currentPosition) + KD*(desiredVelocity - J*thed))

impedanceControl1, impedanceControl2 = impedanceControl 

Save/Pickle variables

In [49]:
stuff = {
    "thdotdot1": thedd[0],
    "thdotdot2": thedd[1],
    "impedanceControl1": impedanceControl1,
    "impedanceControl2": impedanceControl2,
    "gravityCompensation1": Tformulas[0],
    "gravityCompensation2": Tformulas[1]
}

with open(filename, 'wb') as f:
    pickle.dump(stuff, f)

Load/Unpickle variables

In [50]:
with open(filename, 'rb') as f:
    stuff = pickle.load(f)
    
thedd = [ stuff['thdotdot1'], stuff['thdotdot2'] ]
impedanceControl1 = stuff['impedanceControl1']
impedanceControl2 = stuff['impedanceControl2']
Tformulas = [stuff['gravityCompensation1'], stuff['gravityCompensation2']]


Turn formulas into functions

In [61]:
thdotdot1_f = smp.lambdify( (Fx, Fy, T1, T2, g, M1, M2, L1, L2, th1, th2, th1d, th2d) , thedd[0])
thdotdot2_f = smp.lambdify( (Fx, Fy, T1, T2, g, M1, M2, L1, L2, th1, th2, th1d, th2d) , thedd[1])

gravityCompensation1_f = smp.lambdify((Fx, Fy, g, M1, M2, L1, L2, th1, th2, th1d, th2d), Tformulas[0])
gravityCompensation2_f = smp.lambdify((Fx, Fy, g, M1, M2, L1, L2, th1, th2, th1d, th2d), Tformulas[1])

impedanceControl1_f = smp.lambdify((KP, KD, desiredX, desiredY, desiredXdott, desiredYdott, currentX, currentY, L1, L2, th1, th2, th1d, th2d), impedanceControl1)
impedanceControl2_f = smp.lambdify((KP, KD, desiredX, desiredY, desiredXdott, desiredYdott, currentX, currentY, L1, L2, th1, th2, th1d, th2d), impedanceControl2)

Constants

In [62]:
m1 = 1
m2 = 1
l1 = 1
l2 = 1
g_ = 9.8
Kp = 27 #10
Kd = 0 #8

Forward Kinematics

In [63]:
def forwardKin(the1, the2):
    x = - l2*np.sin(the1 + the2) - l1*np.sin(the1)
    y = l2*np.cos(the1 + the2) + l1*np.cos(the1)
    return x, y

def forwardElbowKin(the1):
    x = -l1*np.sin(the1)
    y = l1*np.cos(the1)
    return x, y

Physics Init

In [65]:
theta1 = np.pi*1/4
theta1dot = 0
theta2 = np.pi*1/4
theta2dot = 0

previous_theta1 = theta1
previous_theta2 = theta2

previous_theta1dot = theta1dot
previous_theta2dot = theta2dot

end_effector_x = 0
end_effector_y = 0
Fx_ = 0
Fy_ = 0
desired_end_effector_x, desired_end_effector_y = forwardKin(theta1, theta2)
print(desired_end_effector_x, desired_end_effector_y )

t = 0
dt = 0.04

def updateTheta(T1_, T2_, dt, theta1, theta2, theta1dot, theta2dot):
    theta1dotdot = thdotdot1_f(Fx_, Fy_, T1_, T2_, g_, m1, m2, l1, l2, theta1, theta2, theta1dot, theta2dot)
    theta2dotdot = thdotdot2_f(Fx_, Fy_, T1_, T2_, g_, m1, m2, l1, l2, theta1, theta2, theta1dot, theta2dot)
    

    theta1dot = theta1dotdot*dt + theta1dot
    theta2dot = theta2dotdot*dt + theta2dot

    theta1 = theta1dot*dt + theta1
    theta2 = theta2dot*dt + theta2
    return theta1, theta2, theta1dot, theta2dot


def impedenceControl(theta1, theta2, theta1dot, theta2dot):
    
    currentX, currentY = forwardKin(theta1, theta2)
    
    desiredX = desired_end_effector_x
    desiredY = desired_end_effector_y
    desiredXdott = 0
    desiredYdott = 0
    
    
    impedanceControl1 = impedanceControl1_f(Kp, Kd, desiredX*0.7, desiredY*0.7, desiredXdott, desiredYdott, currentX, currentY, l1, l2, theta1, theta2, theta1dot, theta2dot)
    impedanceControl2 = impedanceControl2_f(Kp, Kd, desiredX*0.7, desiredY*0.7, desiredXdott, desiredYdott, currentX, currentY, l1, l2, theta1, theta2, theta1dot, theta2dot)
    # gravity compensation
    gravityCompensation1 = gravityCompensation1_f(0, 0, g_, m1, m2, l1, l2, theta1, theta2, theta1dot, theta2dot)
    gravityCompensation2 = gravityCompensation2_f(0, 0, g_, m1, m2, l1, l2, theta1, theta2, theta1dot, theta2dot)
    T1_ = impedanceControl1 + gravityCompensation1
    T2_ = impedanceControl2 + gravityCompensation2
    
    return T1_, T2_
    

-1.7071067811865475 0.7071067811865477


Init OpenCV

In [66]:
import cv2
from IPython.display import clear_output

cv2.destroyAllWindows()


def forceIntroduced(event, x, y, flags, param):
    global desired_end_effector_x, desired_end_effector_y, Fx_, Fy_

    if event==cv2.EVENT_LBUTTONDOWN:
        desired_end_effector_x = x*2/100 - 4
        desired_end_effector_y = -y*2/100 + 4

    if event==cv2.EVENT_RBUTTONDOWN:
        tempx = x*2/100 - 4
        tempy = -y*2/100 + 4
        Fx_ = tempx - end_effector_x
        Fy_ = tempy - end_effector_y

    if event==cv2.EVENT_RBUTTONUP:
        Fx_ = 0
        Fy_ = 0
        
def kpChanged(arg):
    global Kp
    Kp = arg
    clear_output(wait=True)
    print("Kp", Kp, "Kd", Kd)

def kdChanged(arg):
    global Kd
    Kd = arg
    clear_output(wait=True)
    print("Kp", Kp, "Kd", Kd)

    
    
cv2.namedWindow('output', cv2.WINDOW_AUTOSIZE)
cv2.createTrackbar('Kp', 'output', 0, 20, kpChanged)
cv2.createTrackbar('Kd', 'output', 0, 20, kdChanged)

cv2.setTrackbarPos('Kp','output', int(Kp))
cv2.setTrackbarPos('Kd','output', int(Kd))


width = 400
height = 400

def draw(theta1, theta2):
    global end_effector_x, end_effector_y
    
    window = np.zeros((height, width, 3), dtype=np.uint8)
    
    
    xElbow, yElbow = forwardElbowKin(theta1)
    xEndEffector, yEndEffector = forwardKin(theta1, theta2)
    
    elbow = (200 + xElbow*70, 200 - yElbow*70)
    end_effector = (200 + xEndEffector*70, 200 - yEndEffector*70)
    
    #elbow = (200 + l1*np.sin(theta1)*70, 200 - l1*np.cos(theta1)*70)
    #end_effector = (elbow[0] + l2*np.sin(theta1 + theta2)*70, elbow[1] - l2*np.cos(theta1 + theta2)*70)

    elbow = (int(elbow[0]), int(elbow[1]))
    end_effector = (int(end_effector[0]), int(end_effector[1]))
    
    end_effector_x = end_effector[0]*2/100 - 4
    end_effector_y = - end_effector[1]*2/100 + 4

    window = cv2.line(window, (200, 200), elbow, (255, 0, 0), 6)
    window = cv2.line(window, elbow, end_effector, (255, 0, 0), 6)
    
    window = cv2.circle(window, (int((desired_end_effector_x+4)*100/2), int((-desired_end_effector_y+4)*100/2)), 5, (255, 255, 255), 2)
    return window

Kp 27 Kd 0


In [67]:
from math import sqrt

def reset():
    global theta1, theta1dot, theta2, theta2dot, t
    theta1 = np.pi*1/4
    theta1dot = 0
    theta2 = np.pi*1/4
    theta2dot = 0
    t = 0

while True:
    #try:
    T1_, T2_ = impedenceControl(theta1, theta2, theta1dot, theta2dot)
    #T1_ = 0
    #T2_ = 0
    theta1, theta2, theta1dot, theta2dot = updateTheta(T1_, T2_, dt, theta1, theta2, theta1dot, theta2dot)

    window = draw(theta1, theta2)
    clear_output(wait=True)
    print(sqrt( (( (desired_end_effector_x - end_effector_x) / 100 ) **2) + (( (desired_end_effector_y - end_effector_y) / 100 )**2) ))
    t = t + dt
    cv2.imshow('output', window)
    cv2.setMouseCallback('output', forceIntroduced)

    #except Exception as e:
    #    print(e)
    #    reset()

    if cv2.waitKey(1) & 0xFF == ord('r'):
        print("reseting")
        reset()
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

        
cv2.destroyAllWindows()

0.00028022917855370096
